In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertConfig
import torch
import numpy as np

import pickle
from tqdm import tqdm

from collections import Counter
import matplotlib.pyplot as plt

import logging
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

---
# Initialise Tokeniser & Model

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

model.eval()

## Defs convert email text to model input and then model output

In [ ]:
def email_to_ids(email_body):
    return torch.tensor(tokenizer.encode(email_body, add_special_tokens=True))

def email_to_vec(email_body_ids, to_id_first=True):
    if to_id_first:
        input_ids = email_to_ids(email_body)
    else:
        input_ids = email_body_ids
        
    input_ids = input_ids.unsqueeze(0)
    outputs = model(input_ids)
    last_hidden_states = outputs[0][0]
    return last_hidden_states.mean(0)  # mean vs sum

---
# Load emails and apply tokeniser and conversion to model input

In [ ]:
with open("../../w3c-emails/emails.pkl", "rb") as handle:
    emails = pickle.load(handle)

In [ ]:
texts = [e.body_raw for e in emails]

In [ ]:
tok_ids = [email_to_ids(t) for t in tqdm(texts)]
with open("emails_token_ids.pkl", "wb") as handle:
    pickle.dump(tok_ids, handle)
exit()

---
# Cut up too emails longer than 512 tokens & group into batches
(start from here by loading converted emails from pkl)

In [ ]:
with open("emails_token_ids.pkl", "rb") as handle:
    ids = pickle.load(handle)

In [ ]:
cut_up = [id_tens.split(511) for id_tens in ids]

In [ ]:
with torch.no_grad():    
    vecs = [[email_to_vec(v, to_id_first=False) for v in tt] for tt in tqdm(cut_up[:5000])]
    
with open("vectors2.pkl", "wb") as handle:
    pickle.dump(vecs, handle)
    
exit()

In [ ]:
with open("vectors.pkl", "rb") as handle:
    vecs_loaded = pickle.load(handle)

In [ ]:
len(vecs_loaded), len(vecs_loaded[0]), vecs_loaded[0][0].shape

In [ ]:
lens = [len(t) for t in texts if len(t) < ]

plt.hist(lens, bins=30)

